# Gradient descent solutions to most likely hidden state by MLE/MAP estimation

This notebook shows how one can use maximum likelihood estimation (MLE) or maximum a posteriori (MAP) estimation to find the most likely hidden state given some sensory observation via gradient descent instead of using the analytic solution.

==========================================================================

* **Notebook dependencies**:
    * ...

* **Content**: Jupyter notebook accompanying Chapter 2 of the textbook "Fundamentals of Active Inference"

* **Author**: Sanjeev Namjoshi (sanjeev.namjoshi@gmail.com)

* **Version**: 0.1

In [10]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import sys
import torch

from torch.distributions import Normal
from types import SimpleNamespace
from typing import Union

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import create_environment

mpl.style.use("seaborn-deep")

The last notebook examined how to find the most likely hidden state estimate after observing the sensory data based on just using the model likelihood (MLE) or the likelihood and the prior (MAP estimation). The result was an analytic update equation for the hidden state. 

We now turn our attention to the gradient descent solution to this problem which is an iterative optimization based approach. This demonstration also introduces the concept of gradient descent which will be important in the chapters that follow.

First we generate 500 data points under a linear generating function as we have done before.

**Note**: we will be using the `torch` package to perform gradient descent so we will need to be using Torch tensors instead of NumPy arrays.

In [16]:
# Environment parameters
env_params = {
    "beta_0_star" : 3,    # Linear parameter intercept
    "beta_1_star" : 2,    # Linear parameter slope
    "y_star_std"  : 0.5   # Standard deviation of sensory data
}

# Initialize environment and agent
env = create_environment(name="static_linear", params=env_params)

# Generate data for three different x_star values
x_star  = 2                                          # 3 different external states
N       = 500                                        # Number of samples
y       = np.zeros(N)                                # Empty array for i=20 samples

# Generate
for i in range(N):
    y[i] = env.generate(x_star)

To perform gradient descent we need a loss function. We will use the loss function described in the previous chapter. These are different for MLE and MAP.
* MLE will use the negative log likelihood.
* MAP will use the negative log likelihood minus the log prior.

In [18]:
def generating_function(beta_0: float, beta_1: float, x: float) -> float:
    return beta_1 * x + beta_0

def mle_objective(x: float, y: Union[float, np.ndarray]) -> np.ndarray:
    
    # Parameters
    beta_0 = 3      # Linear generating function intercept
    beta_1 = 2      # Linear generating function slope
    var_y  = 0.5    # Likelihood variance
    
    # Linear genearting function
    mu_y   = generating_function(beta_0=beta_0, beta_1=beta_1, x=x)
    
    # Calculate log-likelihood over samples
    likelihood_i = norm.pdf(y, loc=mu_y, scale=np.sqrt(var_y))
    log_likelihood_i = np.log(likelihood_i)
    log_likelihood = log_likelihood_i.sum(axis=0)
    
    return -log_likelihood   
    
def map_objective(x: float, y: Union[float, np.ndarray]) -> np.ndarray:
        
    # Parameters
    beta_0 = 3      # Linear generating function intercept
    beta_1 = 2      # Linear generating function slope
    var_y  = 0.5    # Likelihood variance
    m_x    = 2      # Prior mean
    s_x    = 0.25   # Prior variance
    
    # Linear generating function
    mu_y   = generating_function(beta_0=beta_0, beta_1=beta_1, x=x)
    
    # Calculate log-likelihood over samples
    likelihood_i = norm.pdf(y, loc=mu_y, scale=np.sqrt(var_y))
    log_likelihood_i = np.log(likelihood_i)
    log_likelihood = log_likelihood_i.sum(axis=0)
    
    # Calculate log-prior
    log_prior = norm.pdf(x, loc=m_x, scale=np.sqrt(s_x))
    
    
    return -(log_likelihood + log_prior)

Next we need to perform **gradient descent** over our objective function. Let's refer to the objectives in question as $\mathcal{F}_{MLE}$ and $\mathcal{F}_{MAP}$ for MLE and MAP respectively. The aim is to find the minima of this objective function. This minima is the point where the negative log likelihood of the data is minimized (or the negative log likelihood of the data plus log prior assumptions for MAP). This represents the mode of the posterior. Since we are using Gaussian probability distributions, which are quadratic in construction, we will have a well-behaved objective function with a single minima. For each iteration $j \in \left \{0, \dots, J \right \}$ we perform the following update for MLE or MAP:

$$
    \begin{aligned}
        \textbf{MLE}: \hspace{5mm} x^{(j+1)} &\gets x^{(j)} - \kappa \frac{\partial \mathcal{F}_{MLE}}{\partial x} \\
        \textbf{MAP}: \hspace{5mm} x^{(j+1)} &\gets x^{(j)} - \kappa \frac{\partial \mathcal{F}_{MAP}}{\partial x}
    \end{aligned}
$$

where $\kappa$ (kappa) is a **learning rate**. This tells us that we make iterative updates to our running best guess about the value of $x$ by moving in the direction of steepest descent along our loss function, where steepest descent is calculated from the partial derivative of the loss function with respect to $x$. $\kappa$ tells us how big our step down the gradient will be and we are free to specify this hyperparameter ourselves.

This process has no clear termination so when do we stop? We could pick a hard stopping limit, say 100 iterations. Or, we could specify some tolerance. When the updates to the best guess of $x^{(j)}$ at the j-th iteration does not change within some tolerance we know that the process has essentially "flat-lined" or, more properly, **converged** at a local minima. In our case, since we are using a quadratic loss function, this local minima will be the global minima.

Note that we must also choose our initial value for $x$ before kicking off the process. We could choose this randomly or use some prior information about a best guess of where to start. Gradient descent is very sensitive to both the initialization and learning rate (kappa) and the performance of the algorithm depends highly on our choice. For our simple case these settings do not matter as much.

Below we define the gradient descent function. We will use the `torch` package for this purpose which contains functions for **auto-differentiation**. In other words, this package will allow us to easily calculate the partial derivative of the loss function so we do not have to do it by hand.

In [25]:
def gradient_descent(kappa: float, 
                     n_iterations: int, 
                     x: float, 
                     obj: callable):
    
    print(f"Initializing x at {x}.")
    
    # Initialize empty history arrays
    x_history    = torch.zeros(n_iterations)
    loss_history = torch.zeros(n_iterations)
    
    # Calculate loss at initialization
    loss = obj(x, y)
    
    # Add initialization values to history (j=0)
    x_history[0]    = x
    loss_history[0] = loss
    
    # Turn x into a Torch tensor which is differentiable
    x = torch.tensor(x, requires_grad=True)
    
    for j in range(n_iterations):
        obj_x = obj(x, y)
        obj_x.backward()
        
        # Perform gradient descent
        with torch.no_grad():
            x -= (kappa * x.grad)
            x.grad.zero_()   # Zero out the gradients
        
        # Recalculate loss
        loss = obj(x[0], y)
        
        # Append to history
        x_history[j] = x[0]
        loss_history[j] = loss

    return x_history, loss_history
    
    

In [26]:
gradient_descent(
    kappa = 0.000001, 
    n_iterations = 100, 
    x = 5., 
    obj = mle_objective)

Initializing x at 5.0.


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [27]:
x = torch.tensor([0.5, 0.5])

In [31]:
x[1]

tensor(0.5000)